In [1]:
import pandas as pd
import numpy as np
import datetime
from bs4 import BeautifulSoup
import urllib3
import requests
import time
import re
import os
import math

In [2]:
base_url = "https://pittsburgh.craigslist.org/search/fua"

page = requests.get(base_url)

In [3]:
print(page.content)

b'<!DOCTYPE html>\n<html>\n<head>\n    \n\t<meta charset="UTF-8">\n\t<meta http-equiv="X-UA-Compatible" content="IE=Edge">\n\t<meta name="viewport" content="width=device-width,initial-scale=1">\n\t<meta property="og:site_name" content="craigslist">\n\t<meta name="twitter:card" content="preview">\n\t<meta property="og:title" content="pittsburgh furniture - craigslist">\n\t<meta name="description" content="pittsburgh furniture - craigslist">\n\t<meta property="og:description" content="pittsburgh furniture - craigslist">\n\t<meta property="og:url" content="https://pittsburgh.craigslist.org/search/fua">\n\t<meta name="smartbanner:api" content="true">\n\t<meta name="smartbanner:title" content="the craigslist app">\n\t<meta name="smartbanner:author" content="what&#39;s old is new">\n\t<meta name="smartbanner:icon-apple" content="/images/app_icon.png">\n\t<meta name="smartbanner:icon-google" content="/images/app_icon.png">\n\t<meta name="smartbanner:button" content="view">\n\t<meta name="smar

In [4]:
soup = BeautifulSoup(page.content, "html.parser")

total_results = int(soup.find('span', 'totalcount').text)
print(total_results)

total_pages = (total_results // 120) + 1

search_results = []

3000


In [5]:
# results_row = soup.find(class_="rows")

results = soup.find('ul', {'id': 'search-results'})
results_row = results.find_all('li', 'result-row')

print(results.prettify())

<ul class="rows" id="search-results">
 <li class="result-row" data-pid="7532241474" data-repost-of="7491447171">
  <a class="result-image gallery" data-ids="3:00r0r_a2SDzt8sLlNz_0t20t2,3:00f0f_8bMvD2SC6egz_0t20CI,3:00Z0Z_gEiqsfY45zOz_0t20CI,3:00J0J_ftLyKtHsQ2kz_0t20CI,3:00m0m_hJTObhjDf31z_0t20CI,3:00N0N_hgh2k480WJ3z_0t20CI,3:00G0G_lKuhcwWekMyz_0t20CI,3:00c0c_5tKMTtmJLuaz_0t20CI,3:00k0k_cMd3Dhx1OjVz_0t20CI,3:00v0v_iXWoQ8Pi4Wqz_0t20CI,3:00000_4CsOikH9RmBz_0t20CI,3:00o0o_537aQip7Sftz_0t20t2,3:00M0M_3ZbhwoYwzn1z_0t20t2,3:00909_1MdkDj2pP0uz_0t20t2,3:00N0N_glpIKIbUAARz_0t20t2,3:00808_cUhLM1tr9mnz_0t20t2" href="https://pittsburgh.craigslist.org/fuo/d/pittsburgh-new-36-vanity-bathroom/7532241474.html">
   <span class="result-price">
    $40
   </span>
  </a>
  <div class="result-info">
   <span class="icon icon-star" role="button">
    <span class="screen-reader-text">
     favorite this post
    </span>
   </span>
   <time class="result-date" datetime="2022-09-16 14:31" title="Fri 16 Sep 02:3

In [6]:
# furniture_elems = results.find_all('li', class_="result-row")
# for furniture_elem in furniture_elems:
#     print(furniture_elem.text)
#     url = furniture_elem.find('a', class_="result-image gallery")['href']

# changed to process 5 pages instead of total_pages
for i in range(0, 6):
    
    params = {
        's': i*120        
    }

    response = requests.get(base_url, params=params)
    print('Processing Page {0}'.format(i+1))

    response = requests.get(base_url, params=params)
    soup = BeautifulSoup(response.content, 'html.parser')

    results = soup.find('ul', {'id': 'search-results'})
    result_rows = results.find_all('li', 'result-row')

    for result_row in results_row:
        post_datetime = result_row.time['datetime']
        post_id = result_row.h3.a['data-id']
        post_url = result_row.h3.a['href']
        price = result_row.find('span', 'result-price').text
        location = result_row.find('span', 'result-hood').text if result_row.find('span', 'result-hood') else ''
        post_title = result_row.h3.a.text
    # post_title = furniture_elem.find('a', class_="result-title hdrlnk")

        search_results.append([
            post_datetime, post_id, post_url, price, location, post_title
        ])

        time.sleep(1)

    columns = ('Post Date', 'Post Id', 'Post URL', 'Price', 'Location', 'Post Title')
    df = pd.DataFrame(search_results, columns=columns)
    
#     print(price_elem.text.strip())
#     print(location_elem)
#     print(title_elem)
#     print(url_elem)
#     print(price_elem.text.strip())
#     print()

Processing Page 1
Processing Page 2
Processing Page 3
Processing Page 4
Processing Page 5
Processing Page 6


In [8]:
print(df)

            Post Date     Post Id  \
0    2022-09-16 14:31  7532241474   
1    2022-09-16 14:30  7532232029   
2    2022-09-16 14:29  7534654978   
3    2022-09-16 14:27  7534653922   
4    2022-09-16 14:24  7534652114   
..                ...         ...   
715  2022-09-16 09:43  7529774259   
716  2022-09-16 09:41  7534505055   
717  2022-09-16 09:39  7525307762   
718  2022-09-16 09:37  7524872188   
719  2022-09-16 09:35  7533275893   

                                              Post URL Price         Location  \
0    https://pittsburgh.craigslist.org/fuo/d/pittsb...   $40        (Dormont)   
1    https://pittsburgh.craigslist.org/fuo/d/pittsb...   $30        (Dormont)   
2    https://pittsburgh.craigslist.org/fuo/d/wexfor...  $350        (Wexford)   
3    https://pittsburgh.craigslist.org/fuo/d/alliso...  $225   (Allison Park)   
4    https://pittsburgh.craigslist.org/fuo/d/export...    $0                    
..                                                 ...   ...         

In [9]:
timestamp = datetime.datetime.now().strftime('%m_%d_%y %H%M%S')
df.to_csv(f'Craigslist Results ({timestamp}).csv', index=False)

In [10]:
# FURNITURE_URL = 'https://pittsburgh.craigslist.org/fuo/d/murrysville-lift-recliner/7534553122.html'
# furniture_page = requests.get(FURNITURE_URL)
# furniture_soup = BeautifulSoup(furniture_page.content, 'html.parser')

In [11]:
# attributes = furniture_soup.find_all('p', class_='attrgroup')

In [12]:
# for attribute in attributes:
#     spans = attribute.find_all('span')
#     for span in spans:
#         text = span.text.strip()
#         print(text)

In [13]:
# posting_body = furniture_soup.find('section', {"id":"postingbody"})
# print(posting_body.text)